We start by loading BeautifulSoup

In [124]:
#%load_ext soup
import requests
from bs4 import BeautifulSoup
import pandas as pd
from os import path
import numpy as np

This is the url for getting the public information from IS Acdemia

In [ ]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"

We request the form from ISA in order to parse the necessary arguments with Beautifulsoup

In [ ]:
r = requests.get(url)
isa_form = BeautifulSoup(r.text, 'html.parser')

In [ ]:
request_codes = {}
for input in isa_form.find_all('select'):
    for element in input.find_all('option'):
        request_codes[element.text] = (input['name'], element['value'])

This is an example of how we can use the request codes for getting all students in bachelor semester 1 in informatics in 2007-2008

In [ ]:
arguments = {}
arguments["ww_x_GPS"] = "-1"
arguments["ww_i_reportModel"] = "133685247"
arguments["ww_i_reportModelXsl"] = "133685270"
arguments[request_codes["Informatique"][0]] = request_codes["Informatique"][1]
#arguments[request_codes["2007-2008"][0]] = request_codes["2007-2008"][1]
#arguments[request_codes["Bachelor semestre 1"][0]] = request_codes["Bachelor semestre 1"][1]
arguments["ww_x_HIVERETE"] = "null"

In [ ]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?"
r = requests.get(url, params=arguments)

Do all requests for 2007-2016 for both bach 1, 5 and 6 and save the data in csv format
This also checks if the data already exists it doesn't do the requests again

In [104]:
semesters = [1,5,6]
files = {}
for semester in semesters:
    files[semester] = []
    
for year in range(2007,2017):
    # fill in params request: year
    y = str(year)+"-"+str(year+1)
    arguments[request_codes[y][0]] = request_codes[y][1]
    for semester in semesters:
        # fill in params request: semester
        b = "Bachelor semestre %d" % semester
        arguments[request_codes[b][0]] = request_codes[b][1]
        # do request and save to file
        filename = 'data/bachelor_%d_%d.csv' % (year, semester)
        files[semester].append(filename)
        if not path.isfile(filename):
            r = requests.get(url, params=arguments)
            table = pd.read_html(r.text, flavor='lxml', skiprows=1, header=0)[0]
            table = table.ix[:, [0,1,10]]
            table['year'] = year
            table['semester'] = semester
            table.to_csv(filename, encoding='utf-8')

Read the data from the csv files and combine them into one dataframe for each semester

In [118]:
frames = {}
for semester in semesters:
    frames[semester] = []
    for filename in files[str(semester)]:
        frames[semester].append(pd.read_csv(filename, header=0, encoding='utf-8', index_col=3).ix[:,1:])
    frames[semester] = pd.concat(frames[semester])

Merge the dataframes based on sciper number
This will keep only the students that have both a mention of semester 1 and semester 6. (inner join)
Students that have multiple mentions of semester 6 will appaer multiple times in the dataframe.

In [133]:
frame = pd.merge(frames[1], frames[6], left_index=True, right_index=True).ix[:, [0,1,2,6]]
frame.columns = ['	Civilité', 'Nom Prénom', 'year_sem1', 'year_sem6']
frame.head()

,Civilité,Nom Prénom,year_sem1,year_sem6
No Sciper,,,,
147008,Monsieur,Good Xavier,2008,2010
169569,Monsieur,Arévalo Christian,2007,2009
169731,Monsieur,Knecht Mathieu,2007,2010
169795,Monsieur,Scheiben Pascal,2007,2009
169795,Monsieur,Scheiben Pascal,2007,2010


We will now merge with semester 5 as well because a student can have the following path 1-2-3-4-5-6-5 and graduate after 7 semesters.

In [135]:
frame = pd.merge(frame, frames[5], left_index=True, right_index=True)
#frame.columns = ['	Civilité', 'Nom Prénom', 'year_sem1', 'year_sem6']
frame.head()

,Civilité,Nom Prénom_x,year_sem1,year_x,Civilité,Nom Prénom,year_y,semester
No Sciper,,,,,,,,
147008,Monsieur,Good Xavier,2008,2010,Monsieur,Good Xavier,2010,5
169569,Monsieur,Arévalo Christian,2007,2009,Monsieur,Arévalo Christian,2009,5
169731,Monsieur,Knecht Mathieu,2007,2010,Monsieur,Knecht Mathieu,2010,5
169795,Monsieur,Scheiben Pascal,2007,2009,Monsieur,Scheiben Pascal,2009,5
169795,Monsieur,Scheiben Pascal,2007,2009,Monsieur,Scheiben Pascal,2010,5
